# Discrete Log Proofs
In this section we show two simple proof of discrete log. 
The first one is interactive, and the second is just the result of applying the Fiat Shamir Heuristic to the first. 

We are working in the $\mathbb{GF}(N)$ field, where $N$ is a safe prime of the form $2p + 1$ where $p$ is prime. 
The prover aims to show that he possesses a discrete log base $g$ of $\beta$, i.e. that he knows a $x$ such that $g^x = \beta$. 


## Discrete Log Zero Knowledge Proof

The protocol works as follows:
1. The prover selects a random $r$, and sends to the verifier $\gamma \equiv g^r$
2. The verifier selects a random bit $b$ to the prover
3. The prover send the exponent $y = r + bx$ to the verifier
4. The verifier accepts if $g^y = \gamma \beta^b$ else it rejects  

We start by selecting the safe prime $N$, and by setting the field and $g$

In [24]:
bits = 128

def safe_prime(nbits):
    while True:
        p = random_prime(2^nbits-1, false, 2^(nbits-1))
        if ZZ((p+1)/2).is_prime():
            return p
        
N = safe_prime(bits)
F = FiniteField(N)
g = F.multiplicative_generator()

Now, we select a random $x$ and set $\beta = g^x$. We stress that this $x$ is known only by the prover, while $\beta$ is public

In [25]:
x = randint(1, N - 1)
beta = g^x
beta

313856079567223471556813852363007075062

Now, the prover executes its first step, sending over $\gamma$

In [26]:
r = randint(1, N - 1)
gamma = g^r
gamma

44572679616630252165276673530462144282

The verifier selects a random bit, and sends it over

In [27]:
b = randint(0, 1)
b

1

The prover answers with the exponent $y$

In [28]:
y = r + b * x

Finally, the verifier check that the equation $g^y = \gamma \beta^b$ holds

In [29]:
g^y == gamma * beta^b

True

We clear all our variables to make space for the next proof

In [30]:
del bits, N, F, g, x, beta, r, gamma, b, y

## Discrete Log Non Interactive Zero Knowledge

The following works as the above, but it replaces the interaction of the prover with an application of an hash function. 
In particular:
1. The prover selects a random $r$, and computes $\gamma = g^r$. It then uses $b \equiv H(g, \beta, \gamma)$ as its challenge, computing then $y \equiv r + b x$ as an exponent. It then produces the certificate $(\gamma, y)$ and sends it the verifier
2. The verifier computes $b \equiv H(g, \beta, \gamma)$, and accepts iff $g^y = \gamma \beta^b$


We again start by resetting our variables, and starting to find $N$. We also add a SHA1 based hash function to the mix

In [31]:
bits = 128

import hashlib
from sage.crypto.util import ascii_to_bin

def H(*args):
    hs = hashlib.sha1()
    for a in args:
        hs.update(str(a))
    return hs.hexdigest()

def hash_to_int(hs):
    return ZZ('0x' + hs)

N = safe_prime(bits)
F = FiniteField(N)
g = F.multiplicative_generator()

First of all, we select an $x$ and a $\beta$ as before. Again we stress $x$ is private

In [32]:
x = randint(1, N - 1)
beta = g^x
beta

4521956386145255359612433467866229070

Now the verifier selects a random $r$, and computes $\gamma$. This time it uses the hash of the public (or soon to be) parameters as its challenge. It then proceeds as before. The pair $(\gamma, y)$ works as the certificate that any verifier can check

In [33]:
r = randint(1, N - 1)
gamma = g^r
b_prover = hash_to_int(H(g, beta, gamma))
y = r + b_prover * x

(gamma, y)

(104154498362876310642096537674563894683,
 41277114345439426439317073190866215114829058863910655235855738749569003631037159666298)

Finally, the verifier (or any verifier for that matter) can compute the challenge $b$ from the public parameters and the certificate, it then can check that the equation $g^y = \gamma \beta^b$ holds

In [34]:
b_verifier = hash_to_int(H(g, beta, gamma))
g^y == gamma * beta^b_verifier

True